In [1]:
import requests, os, pandas as pd
from io import StringIO
from multiprocessing import Pool


In [2]:
INTERVAL = '61'

In [4]:
stock = 'YESBANK'

In [5]:
if not os.path.isfile(stock+'.csv'):
    stockfile = open(stock+'.csv','w')
    stockfile.write('Date,Time,Open,High,Low,Close,Volume\n')
    stockfile.close()

In [7]:
def puller(stock, no_of_days, interval, write_to_file = True):
  
    no_of_days = str(no_of_days)
    interval = str(Interval)
    
    p = requests.get('http://finance.google.com/finance/getprices?q='+stock+'&x=NSE&i='+interval+'&p='+no_of_days+'d&f=d,c,h,l,o,v').text
    a = pd.read_csv(StringIO(p), skiprows=range(7), names = ['date', 'Close', 'High', 'Low', 'Open', 'Volume'])
    
    if write_to_file:
        a['Date'] = pd.to_datetime(a.date.str[1:],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata').dt.strftime('%Y%m%d')
        a['Time'] = pd.to_datetime(a.date.str[1:],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata').dt.strftime('%H%M%S')
    
        a=a[['Date','Time','Open','High','Low','Close','Volume']]
        a.to_csv(stock+'.csv', mode='a', header=False,index=False)
        print(stock)
    else:
        a['date']=pd.to_datetime(a.date.str[1:],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        a['Date']=a.date.dt.date
        a['Time']=a.date.dt.time
        a=a[['Date','Time','Open','High','Low','Close','Volume']]
        return a

In [8]:
if __name__ == '__main__':
  NO_OF_DAYS = 10
  pool = Pool(processes = 10)
  pool.apply_async(func=puller, args=(stock, NO_OF_DAYS, INTERVAL))